In [ ]:
# Week 12 – Trying out Portfolio Calculation (Usage of Data Structures)

This notebook reads the file `portfolio progamming exercise Nov28.csv`, stores the data in a suitable data structure,
and calculates the total cost of buying all shares in the portfolio as specified.

In [12]:
from dataclasses import dataclass
from pathlib import Path
import csv
from typing import List

In [13]:
# A clean data structure for one stock holding as required
@dataclass
class Holding:
    name: str
    shares: int
    price: float

In [14]:
def read_portfolio(filename: str) -> List[Holding]:
    """
    Read a CSV portfolio file and return a list of Holding objects.
    The CSV is expected to have columns: Name, Shares, Price.
    """
    csv_path = Path(filename)
    holdings: List[Holding] = []

    with csv_path.open(newline="") as f:
        rows = csv.reader(f)
        # I am skipping the header row from excel csv file (Name, Shares, Price)
        header = next(rows, None)

        for name, shares, price in rows:
            holding = Holding(
                name=name,
                shares=int(shares),
                price=float(price),
            )
            holdings.append(holding)

    return holdings

In [15]:
def portfolio_cost(holdings: List[Holding]) -> float:
    """
    Return the total cost to buy all shares in the portfolio.
    """
    return sum(h.shares * h.price for h in holdings)

In [19]:
def show_portfolio(holdings: List[Holding]) -> None:
    """
    To print each holding and the total cost.
    """
    print("name  shares    price")
    for h in holdings:
        # :4  -> pad ticker to width 4
        # :6d -> pad integer shares to width 6
        # :7.2f -> pad price to width 7, 2 decimals
        print(f"{h.name:4}  {h.shares:6d}  {h.price:7.2f}")

    total = portfolio_cost(holdings)
    print(f"\nTotal cost: ${total:,.2f}")

In [20]:
# Now we will use the actual filename which has been uploaded in my jupyter.
filename = "portfolio programming exercise Nov28.csv"

portfolio = read_portfolio(filename)
show_portfolio(portfolio)

name  shares    price
AAPL     100   173.93
MSFT      50   319.53
GOOG      80   131.36
AMZN     200   129.33
VNDA      20   410.17
TSLA     150   255.70

Total cost: $116,302.70


In [21]:
#now we run with yfinance over the above exercise

!pip install yfinance

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 156.5 kB/s eta 0:00:00 kB/s eta 0:00:01:03
  Installing bdone
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 221.4 kB/s eta 0:00:001m220.0 kB/s eta 0:00:01
  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=43ed4e27b589e936cdd529dfc455f32d9f6856b1753052b757bd4fdd7eb63898
  Stored in d

In [22]:
import yfinance as yf

In [23]:
def get_current_price(ticker):
    """
    Trying to return the latest closing price for the given ticker symbol
    using yfinance module here - have tried doing this with AI assistance.
    """
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1d")  # last 1 day
    if hist.empty:
        # if nothing came back, just say 0 (or you can raise an error)
        return 0.0
    return float(hist["Close"].iloc[-1])

In [24]:
print("name  shares   buy_price  current_price  original_value  current_value")
print("----  ------   ---------  -------------  -------------  -------------")

total_original = 0.0
total_current = 0.0

for h in portfolio:   # h is Holding as per headings in the excel file (name, shares, price)
    buy_price = h.price
    current_price = get_current_price(h.name)
    
    original_value = h.shares * buy_price
    current_value = h.shares * current_price
    
    total_original += original_value
    total_current += current_value
    
    print(
        f"{h.name:4}  "
        f"{h.shares:6d}  "
        f"{buy_price:9.2f}  "
        f"{current_price:13.2f}  "
        f"{original_value:13.2f}  "
        f"{current_value:13.2f}"
    )

print("\nTotal original value: ${:,.2f}".format(total_original))
print("Total current value:  ${:,.2f}".format(total_current))
print("Net P/L:              ${:,.2f}".format(total_current - total_original))

name  shares   buy_price  current_price  original_value  current_value
----  ------   ---------  -------------  -------------  -------------
AAPL     100     173.93         278.85       17393.00       27885.00
MSFT      50     319.53         492.01       15976.50       24600.50
GOOG      80     131.36         320.12       10508.80       25609.60
AMZN     200     129.33         233.22       25866.00       46644.00
VNDA      20     410.17           5.36        8203.40         107.20
TSLA     150     255.70         430.17       38355.00       64525.50

Total original value: $116,302.70
Total current value:  $189,371.80
Net P/L:              $73,069.10
